# HW Assignment 2

Instructions: Implement both PG and an evolutionary algorithm to solve the Open AI Gym Lunar Lander problem, and then apply it to my area of choice, which is chess.

First, we need to do some setup

In [ ]:
import torch
import numpy as np
import gym

# Set the device
if torch.cuda.is_available():
    device = "gpu" # 🧮
elif torch.backends.mps.is_available():
    device = "mps" # 🧠
else:
    device = "cpu" # 🥺
    
print(f"Using device: {device}")

First, we need to write the code for our Policy Gradient function with a baseline (REINFORCE). I'm going to use PyTorch as my neural network library.

I'm going to start with a basic feed forward net for both the network that chooses the policy and the network that learns states' values.

First, the policy network for choosing actions

In [ ]:
from torch import nn

class PolicyChoice(nn.Module):
    def __init__(self):
        super(PolicyChoice, self).__init__()
        self.flatten = nn.Flatten()
        self.policy = nn.Sequential(
            nn.Linear(8, 8),
            nn.ReLU(),
            nn.Linear(8, 4), # TODO: try reducing to one hidden layer if learning proves initially dificult
            nn.ReLU(),
            nn.Linear(4, 4),
            nn.LogSoftmax() # log softmax for a nice interpretation as probabilities of choosing actions
        )

    def forward(self, x):
        probs = self.policy(x)
        return probs

policy_model = PolicyChoice().to(device)

For our loss function for the policy network, we want to adjust just the parameters with the primary aim of affecting the probability of taking the action that we took on that time step. If the return of the resulting state is better than expected, we want to increase it proportionally. If it is less than expected, we want to decrease it proportionally. Thus, we multiply the gradient of the parameter weights w.r.t. the taken action's probability by the difference of the return for that state-action pair.

Importantly, there is an extra factor however that we must consider; when we decide that we want to take the gradient of the parameters w.r.t. a specific action's return, the policy expectancy must be multiplied by the specific action's likelihood to determine the value it contributes to the policy. Thus, we end up with the gradient of the action's probability conditioned on the state and parameters. 

Thus, the general concept of loss to backpropogate in the REINFORCE algorithm is:


$\Large (G_t - \hat{\upsilon}) \frac{\nabla\pi(A_t|S_t, \theta)}{\pi(A_t|S_t, \theta)}$

This can be expressed as:

$\Large (G_t - \hat{\upsilon}) \nabla \ln{\pi(A_t|S_t, \theta)}$


The code below just worries about the loss and not the gradient, as PyTorch provides autograd differntiation.

In [ ]:
def policy_loss(prob, state_return):
    return np.log(nn.CrossEntropyLoss(prob, torch.ones(1)) * state_return)

Now, the network for approximating state utililities.

In [ ]:
class StateUtility(nn.Module):
    def __init__(self):
        super(StateUtility, self).__init__()
        self.flatten = nn.Flatten()
        self.state_utility = nn.Sequential(
            nn.Linear(8, 8),
            nn.ReLU(),
            nn.Linear(8, 4), # TODO: try reducing to one hidden layer if learning proves initially dificult
            nn.ReLU(),
            nn.Linear(4, 1), # output a tensor of a scalar value
        )

    def forward(self, x):
        state_utility = self.state_utility(x)
        return state_utility

state_util_model = StateUtility().to(device)

For the state utilities network, we just use L1 loss with the gradients of W with respect to state utility.

$\Large (G_t - \hat{\upsilon}(S_t, W)) \nabla \hat{\upsilon}(S_t, W)$

Like above, the code below just worries about the loss and not the gradient, as PyTorch provides autograd differntiation.

In [ ]:
def state_util_loss(overall_state_value, state_value):
    # the overall state value is the input, and the individual state value is our target
    return nn.L1Loss(overall_state_value, state_value)


We also need a function to calculate a state instance's utility in a given episode.

In [ ]:
def calculate_util():

Let's load the Lunar Lander environment now

In [ ]:
# env = gym.make(
#     "LunarLander-v2",
#     continuous = False,
#     gravity = -10.0,
#     enable_wind = False,
#     wind_power = 15.0,
#     turbulence_power = 1.5,
# )

env = gym.make(
    "LunarLander-v2"
)

observation, info = env.reset(seed=13)

for episode in range(10000):
    action = policy(observation)  # User-defined policy function
    observation, reward, terminated, truncated, info = env.step(action)
    if terminated or truncated:
      observation, info = env.reset()
env.close()